In [15]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
import scipy.integrate as spi

from scipy.optimize import brentq, fsolve
from scipy.integrate import quad
import time
import os

import tensorflow as tf
print(tf.__file__)      # should point into site‑packages
print(hasattr(tf, 'constant'))  # True
import tensornetwork as tn

# Set TensorNetwork to use the TensorFlow backend (needed for autodiff)
tn.set_default_backend("tensorflow")


N = 5 #number of qubits

# each site tensor has shape (chi_left=1, d=2, chi_right=1)
# Initialise all the qubits in |0>, i.e., A[0]=1, A[1]=0
prod_tensors = []
for _ in range(N):
    # shape (2,) → reshape into (1,2,1)
    v = tf.constant([1.0, 0.0], dtype=tf.complex64)
    prod_tensors.append(tf.reshape(v, (1,2,1)))

# initialize FiniteMPS (in canonical form)
psi_initial = tn.FiniteMPS(prod_tensors, canonicalize=True)

# Hadamard
H_gate = tf.constant([[1.0,  1.0],
                 [1.0, -1.0]], dtype=tf.complex64) / tf.cast(tf.sqrt(2.0), tf.complex64)

# apply it on the first site by contracting over the physical index:
# new A[0]_{α,s',β} = Σ_s H[s',s] · A[0]_{α,s,β}
A0 = psi_initial.tensors[0] # shape (1,2,1)
A0 = tf.tensordot(H_gate, A0, axes=([1],[1]))  # → (2,1,1)
psi_initial.tensors[0] = tf.transpose(A0, (1,0,2))  


None
False


AttributeError: module 'tensorflow' has no attribute 'constant'

In [ ]:
#Two-qubit unitary
I = tf.constant([
        [1 + 0j, 0 + 0j],
        [0 + 0j,  1 + 0j]
    ], dtype=tf.complex64)
X = tf.constant([
        [0 + 0j, 1 + 0j],
        [1 + 0j,  0 + 0j]
    ], dtype=tf.complex64)
Y = tf.constant([
        [0 + 0j, 0 - 1j],
        [0 + 1j,  0 + 0j]
    ], dtype=tf.complex64)
Z = tf.constant([
        [1 + 0j, 0 + 0j],
        [0 + 0j,  -1 + 0j]
    ], dtype=tf.complex64)

# 15 Hermitian generator basis: 
# 9 two-site interactions + 3 local on qubit 1 + 3 local on qubit 2
paulis = [X, Y, Z]
basis_list = []

# Interaction terms sigma^a ⊗ sigma^b (a,b in {X,Y,Z})
for A in paulis:
    for B in paulis:
        basis_list.append(tf.linalg.LinearOperatorKronecker([tf.linalg.LinearOperatorFullMatrix(A),
                                                              tf.linalg.LinearOperatorFullMatrix(B)])
                          .to_dense())


# Single-qubit rotations on 1: sigma^a ⊗ I
for A in paulis:
    basis_list.append(tf.linalg.LinearOperatorKronecker([tf.linalg.LinearOperatorFullMatrix(A),
                                                          tf.linalg.LinearOperatorFullMatrix(I)])
                      .to_dense())

# Single-qubit rotations on 2: I ⊗ sigma^a
for B in paulis:
    basis_list.append(tf.linalg.LinearOperatorKronecker([tf.linalg.LinearOperatorFullMatrix(I),
                                                          tf.linalg.LinearOperatorFullMatrix(B)])
                      .to_dense())

# Stack into a (15,4,4) tensor
basis = tf.stack(basis_list, axis=0)  # shape (15,4,4)


@tf.function
def make_two_qubit_unitary(theta):
    # theta: real tf.Tensor shape (15,)
    theta_c = tf.cast(theta, tf.complex64)               # → complex64
    H = tf.tensordot(theta_c, basis, axes=([0],[0]))    # shape (4,4)
    U_flat = tf.linalg.expm(-1j * H)                     # shape (4,4)
    return tf.reshape(U_flat, [2,2,2,2])                 # shape (2,2,2,2)



theta = tf.Variable(tf.random.normal([15], dtype=tf.float32))
U_gate = make_two_qubit_unitary(theta)

# U_gate is ready to apply in your two-site update
print("Two-qubit gate shape:", U_gate.shape)

Two-qubit gate shape: (2, 2, 2, 2)


In [ ]:
#Cost function
mpo_tensors = []
for _ in range(N):
    t = tf.reshape(X, (1, 1, 2, 2))   # (phys_in, phys_out, W_left, W_right)
    mpo_tensors.append(t)

mpo_X = FiniteMPO(mpo_tensors) #creates the string X1 X2 ... XN

In [ ]:
psi_initial.expectation_value(mpo_X) 

AttributeError: 'FiniteMPS' object has no attribute 'expectation_value'